In [ ]:
#예제 8.1 cv2.preCornerDetect() 코너점 검출
import cv2
import numpy as np

#1 # findLocalMaxima()는 src에서 Morphology 연산(팽창 & 침식)으로 
# Local maxia의 좌표를 points 배열에 검출하여 반환함 
def findLocalMaxima(src):
    kernel = cv2.getStructuringElement(shape= cv2.MORPH_RECT,
                                        ksize= (11, 11)) #커널을 구조화, MORPH_RECT: 직사각형
    #local max if kernel = None, 3x3 -> # kernel = None을 사용하면 기본적으로 3 x 3 커널이 사용/ # cv2.dilate()로 src에서 rectKernel의 이웃에서 최댓값을 dilate에 계산
    dilate = cv2.dilate(src, kernel)
    localMax = (src == dilate)

    #local min if kernel = None, 3x3 , # cv2.erode()로 src에서 rectKernel의 이웃에서 최솟값을 erode에 계산
    erode = cv2.erode(src, kernel) 
    localMax2 = src > erode # src > erode로 최솟값보다 큰 위치를 localMax2에 계산
    localMax &= localMax2   # & 연산으로 local maxima 위치를 계산
    points = np.argwhere(localMax == True)  # 행,열 순서로 localMax 위치를 찾음
    points[:,[0, 1]] = points[:,[1, 0]] # switch x, y
    return points

#2
src = cv2.imread('0205.png\cv2.preCornerDetect().jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.preCornerDetect(gray, ksize= 3) # grayscale의 gray에서 cv2.preCornerDetect()로 res를 계산
ret, res2 = cv2.threshold(np.abs(res), 0.1, 0, cv2.THRESH_TOZERO)# findLocalMaxima() 함수를 통해 res2에서 지역 극값의 좌표를 코너점으로 찾아 corners에 저장
corners = findLocalMaxima(res2) # corner를 저장
print('corners.shape=', corners.shape)

#3 # src를 dst에 복사하고, 코너점 배열 corners의 각 코너점 좌표에 cv2.circle()로 dst에 반지름 5, 빨간색 원을 그림
dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x, y), 5, (0, 0, 255), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 8.2 코너점 검출:cornerEigenValsAndVecs()
import cv2
import numpy as np
#1
src = cv2.imread('0205.png\\rectangle2.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerEigenValsAndVecs(gray, blockSize= 5, ksize= 3)
eigen = cv2.split(res)

#2
T = 0.2
ret, edge = cv2.threshold(eigen[0], T, 255, cv2.THRESH_BINARY)
edge = edge.astype(np.uint8)

#3
corners = np.argwhere(eigen[1] > T)
corners[:, [0, 1]] = corners[:, [1, 0]] #switch x, y
print('len(corners) =', len(corners))

dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x, y), 5, (0, 0, 255), 2)

cv2.imshow('edge', edge)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 8.3 코너점 검출1:cv2.cornerMinEigenVal()
import cv2
import numpy as np

#1 
src = cv2.imread('0205.png\cv2.preCornerDetect().jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
eigen = cv2.cornerMinEigenVal(gray, blockSize= 5)
print('eigen.shape=', eigen.shape)

#2 
T = 0.2
corners = np.argwhere(eigen > T)
corners[:, [0, 1]] = corners[:, [1, 0]] #switch x, y
print('len(corners) =', len(corners))
dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x, y), 3, (0, 0, 255), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 8.4 코너점 검출2: cv2.cornerHarris(), cv2.cornerSubPix()
import cv2
import numpy as np

#1 모폴로지 연산으로 지역 극대값의 좌표를 points를 검출하여 반환
def findLocalMaxima(src):
    kernel = cv2.getStructuringElement(shape = cv2.MORPH_RECT,
                                       ksize = (11, 11))
    dilate = cv2.dilate(src,kernel) # local max if kernel = None, 3x3
    localMax = (src == dilate)

    erode = cv2.erode(src,kernel)   # local min if kernel = None, 3x3
    localMax2 = src > erode
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:, [0, 1]] = points[:, [1, 0]]   #switch x, y
    return points
#2
src = cv2.imread('0205.png\cv2.preCornerDetect().jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerHarris(gray, blockSize= 5, ksize= 3, k = 0.01)
ret, res = cv2.threshold(np.abs(res), 0.02, 0, cv2.THRESH_TOZERO)
res8 = cv2.normalize(res, None, 0, 255, cv2.NORM_MINMAX,
                     dtype= cv2.CV_8U)
cv2.imshow('res8', res8)

corners = findLocalMaxima(res)
print('corners=', corners)

#3
#corners = np.float32(corners).copy()
corners = corners.astype(np.float32, order = 'C')
term_crit = (cv2.TERM_CRITERIA_MAX_ITER + 
             cv2.TERM_CRITERIA_EPS, 10, 0.01)
corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1),
                            term_crit)
print('corner2=', corners2)

dst = src.copy()
for x, y in np.int32(corners2):
    cv2.circle(dst, (x, y), 3, (0, 0, 255), 2)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 8.5 코너점 검출2: cv2.cornerHarris(), cv2.cornerSubPix()
import cv2
import numpy as np

#1 
src = cv2.imread('0205.png\cv2.preCornerDetect().jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerHarris(gray, blockSize= 5, ksize= 3, k = 0.01)

#2
res = cv2.dilate(res, None)     #3x3 rect kernel
ret, res = cv2.threshold(res, 0.01 * res.max(), 255,
                         cv2.THRESH_BINARY)
res8 = np.uint8(res)
cv2.imshow('res8', res8)

#3
ret, labels, stats, centroids = \
    cv2.connectedComponentsWithStats(res8)
print('centroids.shape=', centroids.shape)
print('centroids=', centroids)
centroids = np.float32(centroids)

#4
term_crit = (cv2.TERM_CRITERIA_MAX_ITER + 
             cv2.TERM_CRITERIA_EPS, 10, 0.001)
corners = cv2.cornerSubPix(gray, centroids, (5, 5), (-1, -1),
                           term_crit)
print('corners=', corners)

#5
corners = np.round(corners)
dst = src.copy()
for x, y in corners[1:]:
    cv2.circle(dst, (int(x), int(y)), 5, (0, 0, 255), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 8.6 cv2.goodFearuesToTrack() 코너점 검출
import cv2
import numpy as np

#1 
src = cv2.imread('0205.png\cv2.preCornerDetect().jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

K = 30
##K = 10
corners = cv2.goodFeaturesToTrack(gray, maxCorners= K,
                        qualityLevel= 0.05, minDistance= 10)
print('corners.shape=', corners.shape)
print('corners=', corners)

#2
corners2 = cv2.goodFeaturesToTrack(gray, maxCorners= K,
                            qualityLevel= 0.05, minDistance= 10,
                            useHarrisDetector= True, k = 0.04)
print('corners2.shape=', corners2.shape)
print('corners2=', corners2)

#3
dst = src.copy()
corners = corners.reshape(-1, 2)
for x, y in corners:
    cv2.circle(dst, (int(x), int(y)), 5, (0, 0, 255), -1)

corners2 = corners2.reshape(-1, 2)
for x, y in corners2:
    cv2.circle(dst, (int(x), int(y)), 5, (255, 0, 0), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()